# Module 0：環境與基本工具

## 學習目標
1. 確認 PyTorch + CUDA 環境正常運作
2. 掌握 Tensor 基本操作
3. 理解 Autograd（自動微分）機制
4. 完成第一個 GPU 上的線性回歸實作

---

## Part 1：環境驗證

首先確認你的 PyTorch 和 CUDA 環境設定正確。

In [ ]:
import torch
import numpy as np

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: CUDA not available, will use CPU only")

In [ ]:
# 設定 device - 這是之後所有程式都會用到的標準寫法
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

---

## Part 2：Tensor 基礎

### 2.1 什麼是 Tensor？

**直覺理解：** Tensor 就是「多維陣列」，是深度學習中所有資料的載體。

| 維度 | 名稱 | 例子 |
|------|------|------|
| 0D | 純量 (Scalar) | 單一數值，如 `3.14` |
| 1D | 向量 (Vector) | `[1, 2, 3]` |
| 2D | 矩陣 (Matrix) | 灰階圖片 |
| 3D | 3階張量 | RGB 彩色圖 `[C, H, W]` |
| 4D | 4階張量 | 一批 RGB 圖 `[B, C, H, W]` |

**為什麼不直接用 NumPy？**
- PyTorch Tensor 可以在 GPU 上運算（快 10-100 倍）
- PyTorch Tensor 支援自動微分（autograd），這是訓練神經網路的關鍵

### 2.2 建立 Tensor 的各種方法

In [ ]:
# 方法 1：從 Python list 建立
t1 = torch.tensor([1, 2, 3, 4])
print(f"從 list 建立: {t1}")
print(f"  shape: {t1.shape}, dtype: {t1.dtype}")

# 方法 2：從 NumPy 建立（共享記憶體，修改一方會影響另一方）
np_arr = np.array([1.0, 2.0, 3.0])
t2 = torch.from_numpy(np_arr)
print(f"\n從 NumPy 建立: {t2}")

# 方法 3：指定 shape 建立特殊 tensor
t_zeros = torch.zeros(2, 3)      # 全零
t_ones = torch.ones(2, 3)        # 全一
t_rand = torch.rand(2, 3)        # 均勻分佈 [0, 1)
t_randn = torch.randn(2, 3)      # 標準常態分佈 N(0, 1)
t_eye = torch.eye(3)             # 單位矩陣

print(f"\nzeros(2,3):\n{t_zeros}")
print(f"\nrand(2,3):\n{t_rand}")
print(f"\neye(3):\n{t_eye}")

In [ ]:
# 方法 4：用 _like 建立相同 shape 的 tensor
x = torch.tensor([[1, 2], [3, 4]])
x_zeros = torch.zeros_like(x)    # 相同 shape 的全零
x_rand = torch.rand_like(x, dtype=torch.float32)  # 相同 shape 的隨機值

print(f"原始 x:\n{x}")
print(f"\nzeros_like(x):\n{x_zeros}")
print(f"\nrand_like(x):\n{x_rand}")

### 2.3 Tensor 屬性

In [ ]:
t = torch.randn(3, 4, 5)

print(f"Tensor shape: {t.shape}")           # 或 t.size()
print(f"Tensor ndim: {t.ndim}")             # 維度數量
print(f"Tensor dtype: {t.dtype}")           # 資料型別
print(f"Tensor device: {t.device}")         # 在 CPU 還是 GPU
print(f"Total elements: {t.numel()}")       # 元素總數 = 3*4*5 = 60

### 2.4 Tensor 運算

In [ ]:
a = torch.tensor([[1., 2.], [3., 4.]])
b = torch.tensor([[5., 6.], [7., 8.]])

# 逐元素運算 (element-wise)
print("逐元素加法 a + b:")
print(a + b)

print("\n逐元素乘法 a * b:")
print(a * b)

# 矩陣乘法 (matrix multiplication)
print("\n矩陣乘法 a @ b:")
print(a @ b)  # 等同於 torch.matmul(a, b)

# 常用數學函數
print(f"\na.sum() = {a.sum()}")
print(f"a.mean() = {a.mean()}")
print(f"a.max() = {a.max()}")
print(f"a.argmax() = {a.argmax()}")  # 最大值的 index

In [ ]:
# 沿特定維度運算
x = torch.tensor([[1., 2., 3.],
                  [4., 5., 6.]])

print(f"x shape: {x.shape}")
print(f"x:\n{x}")

# dim=0 表示沿著第 0 維（row 方向）壓縮，結果 shape = (3,)
print(f"\nsum(dim=0): {x.sum(dim=0)}  # 每個 column 加總")

# dim=1 表示沿著第 1 維（column 方向）壓縮，結果 shape = (2,)
print(f"sum(dim=1): {x.sum(dim=1)}  # 每個 row 加總")

### 2.5 Reshape 與 View

**重點：** `view` 和 `reshape` 都可以改變 tensor 形狀，但元素總數必須相同。

In [ ]:
x = torch.arange(12)  # [0, 1, 2, ..., 11]
print(f"原始 x: {x}, shape: {x.shape}")

# reshape 成 3x4 矩陣
x_3x4 = x.view(3, 4)
print(f"\nview(3, 4):\n{x_3x4}")

# 用 -1 讓 PyTorch 自動計算該維度
x_2x6 = x.view(2, -1)  # 2 x ? = 12, 所以 ? = 6
print(f"\nview(2, -1):\n{x_2x6}")

# 攤平成 1D
x_flat = x_3x4.flatten()
print(f"\nflatten(): {x_flat}")

### 2.6 CPU / GPU 之間移動資料

**重點：** 運算時，所有 tensor 必須在同一個 device 上！

In [ ]:
# 建立在 CPU 上
x_cpu = torch.randn(3, 3)
print(f"x_cpu device: {x_cpu.device}")

# 移動到 GPU
if torch.cuda.is_available():
    x_gpu = x_cpu.to(device)  # 或 x_cpu.cuda()
    print(f"x_gpu device: {x_gpu.device}")
    
    # 直接在 GPU 上建立
    y_gpu = torch.randn(3, 3, device=device)
    print(f"y_gpu device: {y_gpu.device}")
    
    # GPU tensor 運算
    z_gpu = x_gpu @ y_gpu
    print(f"\nGPU 矩陣乘法結果:\n{z_gpu}")
    
    # 移回 CPU（例如要轉成 NumPy 時必須先移回 CPU）
    z_cpu = z_gpu.cpu()
    z_numpy = z_cpu.numpy()
    print(f"\n轉成 NumPy:\n{z_numpy}")

---

## Part 3：Autograd（自動微分）

### 3.1 為什麼需要自動微分？

**直覺：** 訓練神經網路的核心是「調整參數讓 loss 變小」。怎麼知道參數該往哪個方向調？答案是**梯度（gradient）**。

- 梯度告訴你：「如果我把這個參數稍微增加一點，loss 會增加還是減少？」
- **反向傳播（Backpropagation）** 就是計算所有參數梯度的演算法
- PyTorch 的 `autograd` 會自動幫你做這件事！

### 3.2 requires_grad 與 backward()

In [ ]:
# 建立一個需要追蹤梯度的 tensor
x = torch.tensor([2.0, 3.0], requires_grad=True)
print(f"x = {x}")
print(f"x.requires_grad = {x.requires_grad}")

# 進行一些運算
y = x ** 2          # y = [4, 9]
z = y.sum()         # z = 13
print(f"\ny = x^2 = {y}")
print(f"z = sum(y) = {z}")

# 反向傳播：計算 dz/dx
z.backward()

# 梯度存在 x.grad 裡
# 數學上：z = x0^2 + x1^2，所以 dz/dx0 = 2*x0 = 4，dz/dx1 = 2*x1 = 6
print(f"\nx.grad = dz/dx = {x.grad}")

### 3.3 Computational Graph（計算圖）

**直覺：** PyTorch 在你做運算時，會偷偷記錄「這個值是怎麼算出來的」，形成一張計算圖。

當你呼叫 `backward()` 時，PyTorch 就沿著這張圖「反向」走，用 **chain rule（鏈鎖律）** 計算每個節點的梯度。

```
x ──> y = x^2 ──> z = sum(y)
      │              │
      │   backward   │
      <──────────────┘
      dz/dx = dz/dy * dy/dx
```

In [ ]:
# 更複雜的例子：線性函數
# 模擬 y = w*x + b 的梯度計算

w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)
x = torch.tensor([3.0])  # 輸入，不需要梯度

# Forward pass
y = w * x + b  # y = 2*3 + 1 = 7

# 假設真實值是 10，計算 MSE loss
target = torch.tensor([10.0])
loss = (y - target) ** 2  # loss = (7-10)^2 = 9

print(f"y = {y.item():.2f}")
print(f"loss = {loss.item():.2f}")

# Backward pass
loss.backward()

print(f"\nw.grad = d(loss)/dw = {w.grad.item():.2f}")
print(f"b.grad = d(loss)/db = {b.grad.item():.2f}")

# 手算驗證：
# loss = (wx + b - target)^2
# d(loss)/dw = 2*(wx + b - target) * x = 2*(7-10)*3 = -18
# d(loss)/db = 2*(wx + b - target) * 1 = 2*(7-10)*1 = -6

### 3.4 重要注意事項

In [ ]:
# 注意 1：梯度會累加！每次 backward 前要清零
w = torch.tensor([2.0], requires_grad=True)

for i in range(3):
    y = (w * 3).sum()
    y.backward()
    print(f"Iteration {i+1}: w.grad = {w.grad}")

print("\n梯度累加了！應該每次都是 3，但變成 3, 6, 9")

In [ ]:
# 正確做法：每次 backward 前清零
w = torch.tensor([2.0], requires_grad=True)

for i in range(3):
    if w.grad is not None:
        w.grad.zero_()  # 清零！底線結尾表示 in-place 操作
    
    y = (w * 3).sum()
    y.backward()
    print(f"Iteration {i+1}: w.grad = {w.grad}")

In [ ]:
# 注意 2：推理時不需要梯度，用 torch.no_grad() 節省記憶體
w = torch.tensor([2.0], requires_grad=True)

# 訓練時
y_train = w * 3
print(f"訓練時 y.requires_grad = {y_train.requires_grad}")

# 推理時
with torch.no_grad():
    y_infer = w * 3
    print(f"推理時 y.requires_grad = {y_infer.requires_grad}")

---

## Part 4：nn.Module, Optimizer, Loss

### 4.1 用 nn.Module 定義模型

**直覺：** `nn.Module` 是所有神經網路的基礎類別。你的模型就是一個 Module，裡面可以包含其他 Module（像積木一樣堆疊）。

In [ ]:
import torch.nn as nn

# 最簡單的線性模型：y = wx + b
class SimpleLinear(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        # nn.Linear 包含 weight 和 bias，會自動設定 requires_grad=True
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        # 定義前向傳播：輸入 x 經過什麼運算得到輸出
        return self.linear(x)

# 建立模型
model = SimpleLinear(input_dim=3, output_dim=1)
print(model)

# 查看模型參數
print("\n模型參數：")
for name, param in model.named_parameters():
    print(f"  {name}: shape={param.shape}, requires_grad={param.requires_grad}")

In [ ]:
# 測試前向傳播
x = torch.randn(5, 3)  # 5 個樣本，每個 3 維
y = model(x)           # 等同於 model.forward(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {y.shape}")
print(f"\nOutput:\n{y}")

### 4.2 Loss Function（損失函數）

**直覺：** Loss 衡量「模型預測值」和「真實值」之間的差距。訓練的目標就是讓 loss 越小越好。

常見 loss：
- `nn.MSELoss()`：回歸任務（預測連續值）
- `nn.CrossEntropyLoss()`：分類任務（預測類別）

In [ ]:
# MSE Loss 範例
mse_loss = nn.MSELoss()

pred = torch.tensor([1.0, 2.0, 3.0])
target = torch.tensor([1.5, 2.5, 3.5])

loss = mse_loss(pred, target)
print(f"Predictions: {pred}")
print(f"Targets: {target}")
print(f"MSE Loss: {loss.item():.4f}")

# 手算驗證：MSE = mean((1-1.5)^2 + (2-2.5)^2 + (3-3.5)^2) = mean(0.25+0.25+0.25) = 0.25
print(f"手算驗證: {((pred - target) ** 2).mean().item():.4f}")

### 4.3 Optimizer（優化器）

**直覺：** Optimizer 負責「根據梯度更新參數」。最基本的是 SGD（隨機梯度下降）：

```
new_param = old_param - learning_rate * gradient
```

常見 optimizer：
- `torch.optim.SGD`：最基本
- `torch.optim.Adam`：最常用，自動調整學習率

In [ ]:
import torch.optim as optim

# 建立模型和優化器
model = SimpleLinear(input_dim=3, output_dim=1)
optimizer = optim.SGD(model.parameters(), lr=0.01)  # lr = learning rate

print("優化器會更新這些參數：")
for param_group in optimizer.param_groups:
    print(f"  learning rate: {param_group['lr']}")
    print(f"  參數數量: {len(param_group['params'])}")

### 4.4 訓練循環的標準模式

In [ ]:
# 標準訓練流程（偽代碼）
"""
for epoch in range(num_epochs):
    for x_batch, y_batch in dataloader:
        # 1. 清零梯度
        optimizer.zero_grad()
        
        # 2. 前向傳播
        pred = model(x_batch)
        
        # 3. 計算 loss
        loss = loss_fn(pred, y_batch)
        
        # 4. 反向傳播（計算梯度）
        loss.backward()
        
        # 5. 更新參數
        optimizer.step()
"""
print("標準訓練流程的五個步驟：")
print("1. optimizer.zero_grad()  # 清零梯度")
print("2. pred = model(x)        # 前向傳播")
print("3. loss = loss_fn(pred, y) # 計算損失")
print("4. loss.backward()        # 反向傳播")
print("5. optimizer.step()       # 更新參數")

---

## Part 5：完整範例 - GPU 上的線性回歸

現在把所有東西組合起來，在 GPU 上訓練一個簡單的線性回歸模型。

**任務：** 學習 `y = 2*x1 + 3*x2 - 1` 這個關係

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# 設定 device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 設定隨機種子（確保結果可重現）
torch.manual_seed(42)

In [ ]:
# 生成合成資料
# 真實關係：y = 2*x1 + 3*x2 - 1 + noise

n_samples = 1000
X = torch.randn(n_samples, 2)  # 1000 個樣本，每個 2 維
true_w = torch.tensor([[2.0], [3.0]])
true_b = torch.tensor([-1.0])
noise = torch.randn(n_samples, 1) * 0.1  # 加一點噪音

y = X @ true_w + true_b + noise

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\n真實參數：w = {true_w.squeeze().tolist()}, b = {true_b.item()}")

In [ ]:
# 把資料移到 GPU
X = X.to(device)
y = y.to(device)

print(f"X device: {X.device}")
print(f"y device: {y.device}")

In [ ]:
# 定義模型
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)  # 2 個輸入，1 個輸出
    
    def forward(self, x):
        return self.linear(x)

# 建立模型並移到 GPU
model = LinearRegression().to(device)
print(model)
print(f"\n模型在 device: {next(model.parameters()).device}")

In [ ]:
# 定義 loss 和 optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 訓練前的參數
print("訓練前的參數：")
print(f"  weight: {model.linear.weight.data}")
print(f"  bias: {model.linear.bias.data}")

In [ ]:
# 訓練！
num_epochs = 100
losses = []

for epoch in range(num_epochs):
    # 1. 清零梯度
    optimizer.zero_grad()
    
    # 2. 前向傳播
    pred = model(X)
    
    # 3. 計算 loss
    loss = criterion(pred, y)
    
    # 4. 反向傳播
    loss.backward()
    
    # 5. 更新參數
    optimizer.step()
    
    # 記錄 loss
    losses.append(loss.item())
    
    # 每 20 個 epoch 印一次
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}")

In [ ]:
# 訓練後的參數
print("訓練後的參數：")
print(f"  weight: {model.linear.weight.data}")
print(f"  bias: {model.linear.bias.data}")
print(f"\n真實參數：")
print(f"  weight: {true_w.squeeze().tolist()}")
print(f"  bias: {true_b.item()}")

In [ ]:
# 畫 loss 曲線
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.grid(True)
plt.show()

---

## 練習題（已完成，請閱讀理解）

以下練習題已經完成，包含詳細的 hints 說明。請仔細閱讀程式碼和註解，理解每個步驟的意義。

### 練習 1：改變學習率 (Learning Rate)

**目標：** 觀察不同學習率對訓練的影響

**Hint：**
- 學習率太小：收斂很慢，需要更多 epoch
- 學習率太大：可能震盪甚至發散（loss 變大或 NaN）
- 通常從 0.01 或 0.001 開始嘗試

In [ ]:
# 練習 1：比較不同學習率

def train_with_lr(lr, num_epochs=100):
    """用指定學習率訓練，返回 loss 歷史"""
    # 重新生成資料（確保公平比較）
    torch.manual_seed(42)
    X = torch.randn(1000, 2, device=device)
    y = X @ torch.tensor([[2.0], [3.0]], device=device) + (-1.0) + torch.randn(1000, 1, device=device) * 0.1
    
    # 重新建立模型
    model = LinearRegression().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    losses = []
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    return losses

# 測試不同學習率
learning_rates = [0.001, 0.01, 0.1, 0.5]
results = {}

for lr in learning_rates:
    results[lr] = train_with_lr(lr)
    print(f"LR={lr}: Final loss = {results[lr][-1]:.6f}")

In [ ]:
# 視覺化比較
plt.figure(figsize=(12, 4))

for lr, losses in results.items():
    plt.plot(losses, label=f'LR={lr}')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Rate Comparison')
plt.legend()
plt.grid(True)
plt.yscale('log')  # 用 log scale 更容易看出差異
plt.show()

print("\n觀察：")
print("- LR=0.001：收斂最慢，100 epoch 後還沒完全收斂")
print("- LR=0.01：穩定收斂")
print("- LR=0.1：收斂很快")
print("- LR=0.5：可能會震盪（取決於資料）")

### 練習 2：使用 Adam 優化器

**目標：** 比較 SGD 和 Adam 的差異

**Hint：**
- Adam 會自動調整每個參數的學習率
- Adam 通常更容易調參，對初始學習率沒那麼敏感
- Adam 的預設 lr=0.001 通常就夠用

In [ ]:
# 練習 2：SGD vs Adam

def train_with_optimizer(optimizer_class, lr, num_epochs=100):
    """用指定優化器訓練"""
    torch.manual_seed(42)
    X = torch.randn(1000, 2, device=device)
    y = X @ torch.tensor([[2.0], [3.0]], device=device) + (-1.0) + torch.randn(1000, 1, device=device) * 0.1
    
    model = LinearRegression().to(device)
    criterion = nn.MSELoss()
    optimizer = optimizer_class(model.parameters(), lr=lr)
    
    losses = []
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    return losses

# 比較 SGD 和 Adam（都用 lr=0.01）
sgd_losses = train_with_optimizer(optim.SGD, lr=0.01)
adam_losses = train_with_optimizer(optim.Adam, lr=0.01)

plt.figure(figsize=(10, 4))
plt.plot(sgd_losses, label='SGD (lr=0.01)')
plt.plot(adam_losses, label='Adam (lr=0.01)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('SGD vs Adam')
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.show()

print(f"SGD final loss: {sgd_losses[-1]:.6f}")
print(f"Adam final loss: {adam_losses[-1]:.6f}")
print("\n觀察：Adam 通常在初期收斂更快，對於複雜模型優勢更明顯")

### 練習 3：增加模型複雜度（多層網路）

**目標：** 建立一個多層的神經網路

**Hint：**
- 多層網路需要在層之間加入非線性激活函數（如 ReLU）
- 如果沒有激活函數，多層線性層等於一層線性層（因為線性組合的線性組合還是線性）
- 對於簡單的線性回歸任務，多層網路可能 overkill

In [ ]:
# 練習 3：多層網路（MLP）

class MLP(nn.Module):
    """簡單的多層感知機"""
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        # 定義層
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()  # 激活函數
        self.layer2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.layer1(x)   # 線性變換
        x = self.relu(x)     # 非線性激活
        x = self.layer2(x)   # 線性變換
        return x

# 建立模型
mlp_model = MLP(input_dim=2, hidden_dim=16, output_dim=1).to(device)
print(mlp_model)

# 計算參數數量
total_params = sum(p.numel() for p in mlp_model.parameters())
print(f"\n總參數數量: {total_params}")
print("  - layer1: 2*16 + 16 = 48 (weight + bias)")
print("  - layer2: 16*1 + 1 = 17 (weight + bias)")
print("  - 總共: 48 + 17 = 65")

In [ ]:
# 訓練 MLP
torch.manual_seed(42)
X = torch.randn(1000, 2, device=device)
y = X @ torch.tensor([[2.0], [3.0]], device=device) + (-1.0) + torch.randn(1000, 1, device=device) * 0.1

mlp_model = MLP(input_dim=2, hidden_dim=16, output_dim=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.01)

mlp_losses = []
for epoch in range(100):
    optimizer.zero_grad()
    pred = mlp_model(X)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()
    mlp_losses.append(loss.item())

# 比較單層 vs 多層
linear_losses = train_with_optimizer(optim.Adam, lr=0.01)

plt.figure(figsize=(10, 4))
plt.plot(linear_losses, label='Linear (2 params)')
plt.plot(mlp_losses, label='MLP (65 params)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Linear vs MLP for Linear Regression Task')
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.show()

print(f"Linear final loss: {linear_losses[-1]:.6f}")
print(f"MLP final loss: {mlp_losses[-1]:.6f}")
print("\n觀察：對於線性關係的資料，簡單的線性模型就夠用了")
print("MLP 在這種情況下可能會更慢收斂，因為參數更多")

### 練習 4：使用 DataLoader 做 mini-batch 訓練

**目標：** 學會使用 DataLoader，這是處理大量資料的標準方式

**Hint：**
- DataLoader 會自動把資料分成小批次（mini-batch）
- Mini-batch 訓練比 full-batch 更常見，因為：
  - 節省記憶體（不用一次載入所有資料）
  - 有正則化效果（每個 batch 的梯度有噪音）
  - 可以利用 GPU 並行計算
- 常見 batch size：32, 64, 128, 256

In [ ]:
# 練習 4：DataLoader 使用方式

from torch.utils.data import TensorDataset, DataLoader

# 準備資料
torch.manual_seed(42)
X = torch.randn(1000, 2)
y = X @ torch.tensor([[2.0], [3.0]]) + (-1.0) + torch.randn(1000, 1) * 0.1

# 建立 Dataset 和 DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(
    dataset, 
    batch_size=32,      # 每個 batch 32 個樣本
    shuffle=True,       # 每個 epoch 打亂順序
    num_workers=0,      # 在 notebook 裡用 0，實際訓練可以用 4-8
    pin_memory=True     # 加速 CPU->GPU 資料傳輸
)

print(f"Dataset 大小: {len(dataset)}")
print(f"Batch 大小: {dataloader.batch_size}")
print(f"每個 epoch 的 batch 數: {len(dataloader)}")

In [ ]:
# 看看 DataLoader 怎麼產生 batch
for i, (x_batch, y_batch) in enumerate(dataloader):
    print(f"Batch {i}: x shape = {x_batch.shape}, y shape = {y_batch.shape}")
    if i >= 2:  # 只看前 3 個 batch
        print("...")
        break

In [ ]:
# 使用 DataLoader 訓練
model = LinearRegression().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 20
epoch_losses = []

for epoch in range(num_epochs):
    batch_losses = []
    
    for x_batch, y_batch in dataloader:
        # 把資料移到 GPU
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # 標準訓練流程
        optimizer.zero_grad()
        pred = model(x_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()
        
        batch_losses.append(loss.item())
    
    # 記錄這個 epoch 的平均 loss
    avg_loss = sum(batch_losses) / len(batch_losses)
    epoch_losses.append(avg_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.6f}")

print(f"\n訓練後的參數：")
print(f"  weight: {model.linear.weight.data}")
print(f"  bias: {model.linear.bias.data}")

## Module 0 總結

### 你學到了：

1. **環境驗證**：確認 PyTorch + CUDA 正常運作

2. **Tensor 基礎**：
   - 建立方式：`torch.tensor()`, `torch.zeros()`, `torch.randn()` 等
   - 屬性：`shape`, `dtype`, `device`
   - 運算：逐元素運算 (`+`, `*`)、矩陣乘法 (`@`)
   - CPU/GPU 移動：`.to(device)`, `.cuda()`, `.cpu()`

3. **Autograd**：
   - `requires_grad=True` 追蹤梯度
   - `backward()` 計算梯度
   - 梯度存在 `.grad` 屬性
   - 記得清零梯度！

4. **訓練三元素**：
   - `nn.Module`：定義模型結構
   - `Loss function`：衡量預測與真實值的差距
   - `Optimizer`：根據梯度更新參數

5. **訓練流程**：
   ```python
   optimizer.zero_grad()  # 清零
   pred = model(x)        # 前向
   loss = criterion(pred, y)  # 損失
   loss.backward()        # 反向
   optimizer.step()       # 更新
   ```

---

## Part 6：進階實戰技巧

### 6.1 GPU vs CPU 效能比較

在深度學習中，GPU 的平行運算能力是關鍵優勢。讓我們實際測量一下差異。

In [ ]:
import time

def benchmark_matmul(size, device, num_iterations=100):
    """測量矩陣乘法的效能"""
    # 建立隨機矩陣
    A = torch.randn(size, size, device=device)
    B = torch.randn(size, size, device=device)
    
    # 預熱（讓 GPU 準備好）
    for _ in range(10):
        C = A @ B
    
    if device.type == 'cuda':
        torch.cuda.synchronize()  # 確保 GPU 運算完成
    
    # 計時
    start = time.time()
    for _ in range(num_iterations):
        C = A @ B
    
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    elapsed = time.time() - start
    return elapsed / num_iterations * 1000  # 返回毫秒

# 測試不同大小的矩陣
sizes = [256, 512, 1024, 2048, 4096]
cpu_times = []
gpu_times = []

print("矩陣乘法效能比較 (毫秒/次):")
print("-" * 50)

for size in sizes:
    cpu_time = benchmark_matmul(size, torch.device('cpu'), num_iterations=10)
    cpu_times.append(cpu_time)
    
    if torch.cuda.is_available():
        gpu_time = benchmark_matmul(size, torch.device('cuda'), num_iterations=100)
        gpu_times.append(gpu_time)
        speedup = cpu_time / gpu_time
        print(f"Size {size}x{size}: CPU={cpu_time:.2f}ms, GPU={gpu_time:.3f}ms, Speedup={speedup:.1f}x")
    else:
        print(f"Size {size}x{size}: CPU={cpu_time:.2f}ms, GPU=N/A")

In [ ]:
# 視覺化效能比較
if torch.cuda.is_available() and len(gpu_times) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 絕對時間比較
    x = np.arange(len(sizes))
    width = 0.35
    
    axes[0].bar(x - width/2, cpu_times, width, label='CPU', color='steelblue')
    axes[0].bar(x + width/2, gpu_times, width, label='GPU', color='coral')
    axes[0].set_xlabel('Matrix Size')
    axes[0].set_ylabel('Time (ms)')
    axes[0].set_title('Matrix Multiplication Time')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels([f'{s}x{s}' for s in sizes])
    axes[0].legend()
    axes[0].set_yscale('log')
    
    # 加速比
    speedups = [c/g for c, g in zip(cpu_times, gpu_times)]
    axes[1].bar(x, speedups, color='green', alpha=0.7)
    axes[1].set_xlabel('Matrix Size')
    axes[1].set_ylabel('Speedup (x)')
    axes[1].set_title('GPU Speedup over CPU')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels([f'{s}x{s}' for s in sizes])
    axes[1].axhline(y=1, color='r', linestyle='--', label='Baseline (1x)')
    
    plt.tight_layout()
    plt.show()
    
    print("\n觀察：")
    print("- 矩陣越大，GPU 的優勢越明顯")
    print("- 對於小矩陣，CPU-GPU 資料傳輸的開銷可能抵消 GPU 的速度優勢")
    print("- 這就是為什麼我們用 batch 訓練，而不是一次處理一個樣本")

### 6.2 Broadcasting（廣播機制）

Broadcasting 是 PyTorch 自動擴展 tensor 維度以進行運算的機制。掌握這個可以讓你的代碼更簡潔高效。

In [ ]:
# Broadcasting 範例 1：純量與向量
a = torch.tensor([1, 2, 3])
b = 10  # 純量

print("純量與向量相加:")
print(f"a = {a}")
print(f"a + 10 = {a + b}  # 10 被廣播成 [10, 10, 10]")

# Broadcasting 範例 2：向量與矩陣
A = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])  # shape: (2, 3)
v = torch.tensor([10, 20, 30])  # shape: (3,)

print("\n向量與矩陣相加:")
print(f"A (2x3):\n{A}")
print(f"v (3,): {v}")
print(f"A + v =\n{A + v}  # v 被廣播成 [[10,20,30],[10,20,30]]")

In [ ]:
# Broadcasting 規則詳解
# 規則：從最後一個維度開始比較，兩個維度相容當且僅當：
# 1. 它們相等，或
# 2. 其中一個是 1

# 實用範例：標準化（Z-score）
data = torch.randn(100, 5)  # 100 個樣本，5 個特徵

# 計算每個特徵的均值和標準差
mean = data.mean(dim=0)  # shape: (5,)
std = data.std(dim=0)    # shape: (5,)

# 標準化：(data - mean) / std
# data: (100, 5)
# mean: (5,) -> 廣播成 (100, 5)
normalized = (data - mean) / std

print(f"原始資料 shape: {data.shape}")
print(f"均值 shape: {mean.shape}")
print(f"標準化後 shape: {normalized.shape}")
print(f"\n標準化後每個特徵的均值（應接近 0）: {normalized.mean(dim=0)}")
print(f"標準化後每個特徵的標準差（應接近 1）: {normalized.std(dim=0)}")

In [ ]:
# 進階 Broadcasting：外積計算（Outer Product）
# 這在 attention 機制中很常見

a = torch.tensor([1, 2, 3])  # shape: (3,)
b = torch.tensor([4, 5])     # shape: (2,)

# 外積：每個 a 元素與每個 b 元素相乘
# a.unsqueeze(1): (3,) -> (3, 1)
# b.unsqueeze(0): (2,) -> (1, 2)
# 相乘後: (3, 1) * (1, 2) -> (3, 2)
outer = a.unsqueeze(1) * b.unsqueeze(0)

print(f"a: {a}")
print(f"b: {b}")
print(f"外積 (outer product):\n{outer}")
print(f"\n這等同於:\n{torch.outer(a, b)}")

### 6.3 模型保存與載入

訓練好的模型需要保存下來，以便日後推理或繼續訓練。

In [ ]:
import os

# 建立一個簡單模型
model = LinearRegression().to(device)

# 假裝訓練了一下
with torch.no_grad():
    model.linear.weight.fill_(2.5)
    model.linear.bias.fill_(-1.0)

print("原始模型參數:")
print(f"  weight: {model.linear.weight.data}")
print(f"  bias: {model.linear.bias.data}")

# ========== 方法 1：只保存 state_dict（推薦！） ==========
# state_dict 只包含模型的參數，不包含模型結構
save_path = "model_weights.pth"
torch.save(model.state_dict(), save_path)
print(f"\n✓ 模型參數已保存到 {save_path}")

# 載入：需要先建立模型結構，再載入參數
model_loaded = LinearRegression().to(device)  # 先建立相同結構
model_loaded.load_state_dict(torch.load(save_path, weights_only=True))
model_loaded.eval()  # 切換到評估模式

print("\n載入後的模型參數:")
print(f"  weight: {model_loaded.linear.weight.data}")
print(f"  bias: {model_loaded.linear.bias.data}")

In [ ]:
# ========== 方法 2：保存完整 checkpoint（包含 optimizer 狀態） ==========
# 當你想繼續訓練時，需要保存 optimizer 的狀態

model = LinearRegression().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 模擬訓練幾步
for _ in range(10):
    x = torch.randn(32, 2, device=device)
    y = x @ torch.tensor([[2.0], [3.0]], device=device) + (-1.0)
    
    optimizer.zero_grad()
    pred = model(x)
    loss = nn.MSELoss()(pred, y)
    loss.backward()
    optimizer.step()

# 保存完整 checkpoint
checkpoint = {
    'epoch': 10,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item(),
}
torch.save(checkpoint, 'checkpoint.pth')
print(f"✓ Checkpoint 已保存")
print(f"  包含: epoch, model_state_dict, optimizer_state_dict, loss")

# 載入 checkpoint 繼續訓練
checkpoint = torch.load('checkpoint.pth', weights_only=False)
model_resume = LinearRegression().to(device)
optimizer_resume = optim.Adam(model_resume.parameters(), lr=0.01)

model_resume.load_state_dict(checkpoint['model_state_dict'])
optimizer_resume.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']

print(f"\n✓ 從 epoch {start_epoch} 繼續訓練")
print(f"  上次的 loss: {checkpoint['loss']:.6f}")

### 6.4 常見錯誤與除錯技巧

深度學習程式碼的除錯可能很棘手。以下是一些常見錯誤和解決方法。

In [ ]:
# 錯誤 1：Device 不匹配
print("=" * 50)
print("錯誤 1：Device 不匹配")
print("=" * 50)

try:
    x_cpu = torch.randn(3, 3)
    x_gpu = torch.randn(3, 3, device='cuda') if torch.cuda.is_available() else torch.randn(3, 3)
    
    if torch.cuda.is_available():
        result = x_cpu + x_gpu  # 這會報錯！
except RuntimeError as e:
    print(f"錯誤訊息: {str(e)[:100]}...")
    print("\n解決方法: 確保所有 tensor 在同一個 device")
    print("  x_cpu = x_cpu.to(device)  # 或 x_gpu.cpu()")

In [ ]:
# 錯誤 2：Shape 不匹配
print("=" * 50)
print("錯誤 2：Shape 不匹配")
print("=" * 50)

try:
    a = torch.randn(3, 4)  # shape: (3, 4)
    b = torch.randn(5, 4)  # shape: (5, 4)
    result = a + b  # 這會報錯！
except RuntimeError as e:
    print(f"錯誤訊息: {e}")
    print("\n解決方法: 檢查 tensor 的 shape，可能需要 reshape 或 unsqueeze")
    print("  print(a.shape, b.shape)  # 先檢查 shape")

In [ ]:
# 錯誤 3：NaN 或 Inf 出現（常見於訓練不穩定）
print("=" * 50)
print("錯誤 3：NaN / Inf 出現")
print("=" * 50)

# 模擬 NaN 產生的情況
x = torch.tensor([0.0])
y = torch.log(x)  # log(0) = -inf
z = 0.0 / 0.0     # 0/0 = nan

print(f"log(0) = {y}")
print(f"0/0 = {z}")

# 檢測 NaN 和 Inf 的方法
tensor_with_nan = torch.tensor([1.0, float('nan'), 3.0])
tensor_with_inf = torch.tensor([1.0, float('inf'), 3.0])

print(f"\n檢測 NaN: torch.isnan() -> {torch.isnan(tensor_with_nan)}")
print(f"檢測 Inf: torch.isinf() -> {torch.isinf(tensor_with_inf)}")
print(f"有任何 NaN? {torch.isnan(tensor_with_nan).any()}")

print("\n常見原因和解決方法:")
print("  1. 學習率太大 -> 降低學習率")
print("  2. 數值溢出 -> 使用梯度裁剪 (gradient clipping)")
print("  3. log(0) 或 除以 0 -> 加入 epsilon (如 log(x + 1e-8))")

In [ ]:
# 除錯技巧：使用 hooks 監控中間層
print("=" * 50)
print("除錯技巧：使用 hooks 監控中間層輸出")
print("=" * 50)

# 定義一個簡單的網路
class DebugNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 20)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(20, 5)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 建立模型
debug_model = DebugNet()

# 用來儲存中間輸出的字典
activations = {}

# 定義 hook 函數
def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# 註冊 hooks
debug_model.fc1.register_forward_hook(get_activation('fc1'))
debug_model.relu.register_forward_hook(get_activation('relu'))

# 前向傳播
x = torch.randn(2, 10)
output = debug_model(x)

# 查看中間層輸出
print(f"輸入 shape: {x.shape}")
print(f"fc1 輸出 shape: {activations['fc1'].shape}")
print(f"relu 輸出 shape: {activations['relu'].shape}")
print(f"最終輸出 shape: {output.shape}")
print(f"\nfc1 輸出統計: mean={activations['fc1'].mean():.3f}, std={activations['fc1'].std():.3f}")
print(f"relu 輸出統計: mean={activations['relu'].mean():.3f}, std={activations['relu'].std():.3f}")

### 6.5 GPU 記憶體管理

當你在訓練大型模型時，GPU 記憶體管理變得至關重要。

In [ ]:
if torch.cuda.is_available():
    # 查看 GPU 記憶體使用情況
    print("GPU 記憶體管理工具:")
    print("-" * 50)
    
    # 清空 cache 前
    allocated_before = torch.cuda.memory_allocated() / 1e6
    reserved_before = torch.cuda.memory_reserved() / 1e6
    
    # 建立一些 tensors
    tensors = [torch.randn(1000, 1000, device='cuda') for _ in range(10)]
    
    allocated_during = torch.cuda.memory_allocated() / 1e6
    reserved_during = torch.cuda.memory_reserved() / 1e6
    
    print(f"建立 10 個 1000x1000 tensor 後:")
    print(f"  已分配記憶體: {allocated_during:.1f} MB")
    print(f"  已保留記憶體: {reserved_during:.1f} MB")
    
    # 刪除 tensors
    del tensors
    torch.cuda.empty_cache()  # 釋放未使用的 cache
    
    allocated_after = torch.cuda.memory_allocated() / 1e6
    reserved_after = torch.cuda.memory_reserved() / 1e6
    
    print(f"\n刪除 tensors 並清空 cache 後:")
    print(f"  已分配記憶體: {allocated_after:.1f} MB")
    print(f"  已保留記憶體: {reserved_after:.1f} MB")
    
    # 記憶體摘要
    print(f"\n完整記憶體摘要:")
    print(torch.cuda.memory_summary(abbreviated=True))
else:
    print("CUDA 不可用，跳過 GPU 記憶體管理示範")

### 6.6 混合精度訓練預覽 (Mixed Precision Training)

混合精度訓練使用 FP16 和 FP32 的組合，可以：
- 減少約一半的 GPU 記憶體使用
- 加速訓練（在 Tensor Core GPU 上）
- 通常不會損失模型精度

In [ ]:
# 混合精度訓練範例
from torch.amp import autocast, GradScaler

# 建立模型和資料
model = MLP(input_dim=100, hidden_dim=256, output_dim=10).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 建立 GradScaler（只在 CUDA 上使用）
scaler = GradScaler('cuda') if torch.cuda.is_available() else None

# 模擬資料
x = torch.randn(64, 100, device=device)
y = torch.randint(0, 10, (64,), device=device)

print("混合精度訓練示範:")
print("-" * 50)

# 標準訓練（FP32）
optimizer.zero_grad()
with torch.no_grad():
    start_mem = torch.cuda.memory_allocated() / 1e6 if torch.cuda.is_available() else 0

pred_fp32 = model(x)
loss_fp32 = criterion(pred_fp32, y)
loss_fp32.backward()

with torch.no_grad():
    fp32_mem = torch.cuda.memory_allocated() / 1e6 if torch.cuda.is_available() else 0

print(f"FP32 訓練:")
print(f"  Loss: {loss_fp32.item():.4f}")
if torch.cuda.is_available():
    print(f"  記憶體使用: {fp32_mem:.1f} MB")

In [ ]:
# 混合精度訓練（AMP）
if torch.cuda.is_available():
    model_amp = MLP(input_dim=100, hidden_dim=256, output_dim=10).to(device)
    optimizer_amp = optim.Adam(model_amp.parameters(), lr=0.001)
    scaler = GradScaler('cuda')
    
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    optimizer_amp.zero_grad()
    
    # 使用 autocast 自動轉換精度
    with autocast('cuda'):
        pred_amp = model_amp(x)
        loss_amp = criterion(pred_amp, y)
    
    # 使用 scaler 來縮放 loss，避免 FP16 下溢
    scaler.scale(loss_amp).backward()
    scaler.step(optimizer_amp)
    scaler.update()
    
    amp_mem = torch.cuda.max_memory_allocated() / 1e6
    
    print(f"\n混合精度 (AMP) 訓練:")
    print(f"  Loss: {loss_amp.item():.4f}")
    print(f"  峰值記憶體使用: {amp_mem:.1f} MB")
    
    print("\n混合精度訓練的完整模板:")
    print("""
    scaler = GradScaler('cuda')
    
    for epoch in range(num_epochs):
        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()
            
            with autocast('cuda'):
                pred = model(x_batch)
                loss = criterion(pred, y_batch)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
    """)
else:
    print("CUDA 不可用，跳過混合精度示範")
    print("注意：混合精度訓練主要在 GPU 上使用")

In [ ]:
# 清理臨時檔案
import os
for f in ['model_weights.pth', 'checkpoint.pth']:
    if os.path.exists(f):
        os.remove(f)
        print(f"已刪除臨時檔案: {f}")

---

## 完整總結

### 本 Module 涵蓋的內容：

| 主題 | 重點 |
|------|------|
| **環境設定** | PyTorch + CUDA 驗證、device 設定 |
| **Tensor 基礎** | 建立、屬性、運算、reshape、device 移動 |
| **Autograd** | requires_grad、backward、計算圖、梯度清零 |
| **訓練元件** | nn.Module、Loss、Optimizer |
| **訓練流程** | zero_grad → forward → loss → backward → step |
| **GPU 效能** | CPU vs GPU 比較、記憶體管理 |
| **Broadcasting** | 維度擴展、標準化、外積 |
| **模型保存** | state_dict、checkpoint |
| **除錯技巧** | device 不匹配、shape 錯誤、NaN/Inf、hooks |
| **混合精度** | autocast、GradScaler |

### 實戰 Checklist：

- [ ] 確認 GPU 可用：`torch.cuda.is_available()`
- [ ] 建立標準 device：`device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`
- [ ] 模型和資料都移到同一個 device
- [ ] 訓練前清零梯度：`optimizer.zero_grad()`
- [ ] 推理時停止追蹤梯度：`with torch.no_grad()`
- [ ] 大型模型考慮使用混合精度訓練
- [ ] 定期保存 checkpoint

### 下一步：Module 1 - 深度學習需要的數學基礎

我們將學習：
- 線性代數複習（矩陣運算、特徵值）
- 微積分基礎（偏導數、鏈鎖法則）
- 機率與統計（分佈、最大似然估計）
- 資訊理論（熵、KL散度、交叉熵）